In [1]:
%load_ext autoreload

In [2]:
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display


In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint, RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram, RigidTransform, RollPitchYaw
import meshcat
from pydrake.all import GenerateSeedingPolytope, GenerateRandomSeedingPolytope
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.multibody.rational_forward_kinematics import FindEpsilonLower, FindEpsilonLowerVector, FindEpsilonUpperVector
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import FindMaxEpsForAllIneqs, MosekSolver, MosekSolverDetails,FindMaxEpsTilRedundant
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess


# Build Plant


In [4]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
plant.Finalize()
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7008/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7009/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6008...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7008/static/
Connected to meshcat-server.


In [5]:
# upper = np.array([0.5,0.25, 0.75]) # 0.25*np.ones(3) #
# lower = -upper#np.array([-0.5,-0.75, 0.25]) #-0.25*np.ones(3) #
# inner_box = HPolyhedron.MakeBox(lower, upper)
# trans_list = [inner_box]
# A_list = []
# b_list = []
# num_rot = 2
# num_perm = 2
# seed_point = np.zeros(3)
# seed_poly = GenerateSeedingPolytope(seed_point, num_perm, num_rot)

# # for i in range(num_rot):

# #     theta = 2*np.pi/num_rot *(i)
# #     rot = np.array([
# #         [np.cos(theta), -np.sin(theta), 0],
# #         [np.sin(theta), np.cos(theta), 0],
# #         [0,0,1]])
# #     for j in range(num_perm):
# #         perm = np.eye(3)
# #         perm[:,[0,j]] = perm[:,[j,0]]
# #         P = np.linalg.solve(perm,rot@perm)
# #         print(P)
        
# #         A_list.append(inner_box.A()@P)
# #         b_list.append(inner_box.b())
# #         trans_list.append(HPolyhedron(A_list[-1], b_list[-1]))
# # A = np.vstack(A_list)
# # b = np.hstack(b_list)
# inner = seed_poly.ReduceInequalities()#HPolyhedron(A,b).ReduceInequalities()
# print(inner.A().shape[0]-seed_poly.A().shape[0])

# # visualizer.plot_regions(trans_list, region_suffix = '_all')
# visualizer.plot_regions([seed_poly], region_suffix = '_seed_poly')
# visualizer.plot_regions([inner], region_suffix = '_inner')

In [6]:
# upper = 0.25*np.ones(3) # np.array([0.5,0.25, 0.75]) # 
# lower = -upper#np.array([-0.5,-0.75, 0.25]) #-0.25*np.ones(3) #
# inner_box = HPolyhedron.MakeBox(lower, upper)
# n = 2
# inner_box2 = HPolyhedron(np.vstack([inner_box.A()]*n), np.hstack([inner_box.b()]*n))
# visualizer.plot_regions([inner_box], region_suffix = '_inner_box')

# noise_A = 0.1*np.random.randn(*(inner_box2.A().shape))
# noise_b = 0.1*np.random.randn(*(inner_box2.b().shape))
# inner = HPolyhedron(inner_box2.A() + noise_A, inner_box2.b()+noise_b)
# inner_red = inner.ReduceInequalities()
# visualizer.plot_regions([inner], region_suffix = '_inner')


In [15]:
seed_point = 0.5*np.ones(3)
num_unit_box_copies = 2
initial_box_scale = 0.25
gaussian_variance = 0.1
inner = GenerateRandomSeedingPolytope(seed_point,
                                     num_unit_box_copies,
                                     initial_box_scale,
                                     gaussian_variance)
inner_red = inner.ReduceInequalities()
visualizer.plot_regions([inner], region_suffix = '_inner')
outer = HPolyhedron.MakeUnitBox(3)
visualizer.plot_regions([outer], region_suffix = '_outer', opacity = 0.25)

In [16]:
print(inner.A().shape[0] - inner_red.A().shape[0])


1


In [ ]:
shifted = []
d_all = inner.b()
polys = []
for i in range(inner.A().shape[0]):
    C, d = inner.A(), inner.b()
    prog = MathematicalProgram()
    solver = MosekSolver()
    t = prog.NewContinuousVariables(3, 't')
    c_cost = C[i,:]
    d_cost = d[i]
    C_constraint = np.vstack([C[:i, :], C[(i+1):, :]])
    d_constraint = np.hstack([d[:i], d[(i+1):]])

    t_lower = -np.ones(3)
    t_upper = np.ones(3)

    prog.AddLinearConstraint(C_constraint, -np.inf*np.ones(d_constraint.shape), d_constraint, t)
    prog.AddBoundingBoxConstraint(t_lower, t_upper, t)
    prog.AddLinearCost(-c_cost, d_cost, t)
    result = solver.Solve(prog)
    eps_man = -result.get_optimal_cost()
    eps = FindMaxEpsTilRedundant(c_cost, d_cost, C_constraint, d_constraint, t_lower, t_upper)
    print(eps)
    dtmp = d.copy()
    dtmp[i] += eps
    d_all[i] += eps
    polys.append(HPolyhedron(C,dtmp))
    shifted.append(polys[-1])
shifted.append(HPolyhedron(C,d_all))
visualizer.plot_regions(shifted, region_suffix = '_shifted')

0.11035239106491512
0.22168532589241197
0.02193508583010051
0.014980099191228419
0.05996383509095249
0.09543259633479051
0.02781058847262108
0.025009670914938664
0.07572288419608009
0.1752494793847913
0.08896301585524441
-0.017551260842670363


In [9]:
print(polys[0].A())
print(polys[0].b())

[[ 0.          0.96592583 -0.25881905]
 [ 0.70710678 -0.70710678  0.        ]
 [-0.70710678 -0.70710678  0.        ]
 [-0.70710678  0.70710678  0.        ]
 [ 0.70710678  0.70710678  0.        ]
 [-0.         -1.         -0.        ]
 [ 0.          1.          0.        ]
 [ 0.          0.70710678  0.70710678]
 [ 0.         -0.70710678  0.70710678]
 [ 0.         -0.70710678 -0.70710678]
 [ 0.          0.70710678 -0.70710678]
 [ 0.25881905  0.96592583  0.        ]
 [-0.96592583  0.25881905  0.        ]
 [-0.          0.          1.        ]
 [-0.25881905 -0.96592583  0.        ]
 [ 0.96592583 -0.25881905  0.        ]
 [-0.          0.         -1.        ]
 [ 1.         -0.          0.        ]
 [ 0.          0.25881905  0.96592583]
 [ 0.         -0.96592583  0.25881905]
 [-1.         -0.          0.        ]
 [ 0.         -0.25881905 -0.96592583]]
[0.10731322 0.1        0.1        0.1        0.1        0.1
 0.1        0.1        0.1        0.1        0.1        0.1
 0.1        0.1      

In [10]:
print(d_cost)
print(d_constraint)

0.1
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1]


In [11]:
print(C)
print(d)

[[ 0.          0.96592583 -0.25881905]
 [ 0.70710678 -0.70710678  0.        ]
 [-0.70710678 -0.70710678  0.        ]
 [-0.70710678  0.70710678  0.        ]
 [ 0.70710678  0.70710678  0.        ]
 [-0.         -1.         -0.        ]
 [ 0.          1.          0.        ]
 [ 0.          0.70710678  0.70710678]
 [ 0.         -0.70710678  0.70710678]
 [ 0.         -0.70710678 -0.70710678]
 [ 0.          0.70710678 -0.70710678]
 [ 0.25881905  0.96592583  0.        ]
 [-0.96592583  0.25881905  0.        ]
 [-0.          0.          1.        ]
 [-0.25881905 -0.96592583  0.        ]
 [ 0.96592583 -0.25881905  0.        ]
 [-0.          0.         -1.        ]
 [ 1.         -0.          0.        ]
 [ 0.          0.25881905  0.96592583]
 [ 0.         -0.96592583  0.25881905]
 [-1.         -0.          0.        ]
 [ 0.         -0.25881905 -0.96592583]]
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1]


In [12]:
print(outer.A())
print(outer.b())

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [-1. -0. -0.]
 [-0. -1. -0.]
 [-0. -0. -1.]]
[1. 1. 1. 1. 1. 1.]
